# Calculating dx, dy, dz from other data
*Akira Di Sandro, 7/7/20*
<br> In this notebook, I will be calculating the distance between each i and j points I'm intersted in as well as the actual distance between the depths I'm interested in. I hope to use this data to calculate the surface areas of the grid cells I'm interested in for transport calculations.

## 1. Import Packages as usual

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import xarray as xr
xr.set_options(display_style='html')
import intake
%matplotlib inline
import cartopy.crs as ccrs
import numpy as np
import sectionate
from haversine import haversine, Unit

## 2. Open Dataset

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [4]:
dog = col.search(source_id='GFDL-CM4', experiment_id='historical', table_id='Omon', variable_id=['uo','vo','wo'], grid_label='gn')
dog.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Omon,uo,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...,NaN,20180701
1,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Omon,vo,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...,NaN,20180701


In [5]:
dset_dict = dog.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']

In [6]:
grid = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']
grid

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 35, member_id: 1, time: 1980, x: 1440, y: 1080)
Coordinates:
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * bnds       (bnds) float64 1.0 2.0
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * member_id  (member_id) <U8 'r1i1p1f1'
Data variables:
    uo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
    vo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
Attributes:
    references:              see further_info_url attribute
    forcing_index:           1
    variant_label:           r1i1p1f1
    parent_source_id:        GFDL-CM4
    experiment:              historical
    sub_experiment:          none
    physics_index:           1
    parent_activity_id:      CMIP
    grid_label:              gn
    variant_info:            N/A
    nominal_resolution:      25 km
    mip_era:                 CMIP6
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    parent_mip_era:          CMIP6
    institution_id:          NOAA-GFDL
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    frequency:               mon
    institution:             National Oceanic and Atmospheric Administration,...
    branch_time_in_parent:   36500.0
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    source_type:             AOGCM
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    realm:                   ocean
    tracking_id:             hdl:21.14100/d0ebcd36-d2cd-442e-a8d9-8f7a27168a2...
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    parent_time_units:       days since 0001-1-1
    activity_id:             CMIP
    branch_time_in_child:    0.0
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    history:                 File was processed by fremetar (GFDL analog of C...
    product:                 model-output
    branch_method:           standard
    parent_variant_label:    r1i1p1f1
    contact:                 gfdl.climate.model.info@noaa.gov
    experiment_id:           historical
    comment:                 <null ref>
    table_id:                Omon
    source_id:               GFDL-CM4
    status:                  2019-08-06;created;by nhn2@columbia.edu
    realization_index:       1
    data_specs_version:      01.00.27
    sub_experiment_id:       none
    parent_experiment_id:    piControl
    initialization_index:    1
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

* The above two lines will also be dependent on the change in line 4.

In [7]:
dog1 = col.search(source_id='GFDL-CM4', variable_id=['areacello','volcello'], grid_label='gn')
dog1.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NOAA-GFDL,GFDL-CM4,1pctCO2,r1i1p1f1,Omon,volcello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/1pctCO2/r1i...,NaN,20180701
1,CMIP,NOAA-GFDL,GFDL-CM4,abrupt-4xCO2,r1i1p1f1,Omon,volcello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/abrupt-4xCO...,NaN,20180701
2,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Omon,volcello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...,NaN,20180701
3,CMIP,NOAA-GFDL,GFDL-CM4,piControl,r1i1p1f1,Ofx,areacello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/piControl/r...,NaN,20180701
4,CMIP,NOAA-GFDL,GFDL-CM4,piControl,r1i1p1f1,Omon,volcello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/piControl/r...,NaN,20180701
5,OMIP,NOAA-GFDL,GFDL-CM4,omip1,r1i1p1f1,Ofx,areacello,gn,gs://cmip6/OMIP/NOAA-GFDL/GFDL-CM4/omip1/r1i1p...,NaN,20180701
6,ScenarioMIP,NOAA-GFDL,GFDL-CM4,ssp245,r1i1p1f1,Omon,volcello,gn,gs://cmip6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp2...,NaN,20180701
7,ScenarioMIP,NOAA-GFDL,GFDL-CM4,ssp585,r1i1p1f1,Omon,volcello,gn,gs://cmip6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp5...,NaN,20180701


In [8]:
dset_dict1 = dog1.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict1.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


['CMIP.NOAA-GFDL.GFDL-CM4.piControl.Ofx.gn',
 'OMIP.NOAA-GFDL.GFDL-CM4.omip1.Ofx.gn',
 'CMIP.NOAA-GFDL.GFDL-CM4.abrupt-4xCO2.Omon.gn',
 'ScenarioMIP.NOAA-GFDL.GFDL-CM4.ssp245.Omon.gn',
 'CMIP.NOAA-GFDL.GFDL-CM4.1pctCO2.Omon.gn',
 'ScenarioMIP.NOAA-GFDL.GFDL-CM4.ssp585.Omon.gn',
 'CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn',
 'CMIP.NOAA-GFDL.GFDL-CM4.piControl.Omon.gn']

In [9]:
grid1 = dset_dict1['CMIP.NOAA-GFDL.GFDL-CM4.piControl.Ofx.gn']
grid2 = dset_dict1['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']

In [10]:
grid1

<xarray.Dataset>
Dimensions:    (member_id: 1, vertex: 4, x: 1440, y: 1080)
Coordinates:
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: vertex
Data variables:
    areacello  (member_id, y, x) float32 dask.array<chunksize=(1, 1080, 1440), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    0.0
    branch_time_in_parent:   54750.0
    comment:                 <null ref>
    contact:                 gfdl.climate.model.info@noaa.gov
    creation_date:           2018-11-20T18:41:34Z
    data_specs_version:      01.00.27
    experiment:              pre-industrial control
    experiment_id:           piControl
    forcing_index:           1
    frequency:               fx
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    grid_label:              gn
    history:                 File was processed by fremetar (GFDL analog of C...
    initialization_index:    1
    institution:             National Oceanic and Atmospheric Administration,...
    institution_id:          NOAA-GFDL
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    mip_era:                 CMIP6
    nominal_resolution:      25 km
    parent_activity_id:      CMIP
    parent_experiment_id:    piControl-spinup
    parent_mip_era:          CMIP6
    parent_source_id:        GFDL-CM4
    parent_time_units:       days since 1700-1-1
    parent_variant_label:    r1i1p1f1
    physics_index:           1
    product:                 model-output
    realization_index:       1
    realm:                   ocean
    references:              see further_info_url attribute
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    source_id:               GFDL-CM4
    source_type:             AOGCM
    sub_experiment:          none
    sub_experiment_id:       none
    table_id:                Ofx
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    tracking_id:             hdl:21.14100/82865d31-bc4e-437e-93d7-f37fd704ca65
    variable_id:             areacello
    variant_info:            N/A
    variant_label:           r1i1p1f1
    status:                  2019-10-25;created;by nhn2@columbia.edu
    intake_esm_varname:      areacello
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.piControl.Ofx.gn

In [11]:
grid2

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 35, member_id: 1, time: 1980, vertex: 4, x: 1440, y: 1080)
Coordinates:
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * bnds       (bnds) float64 1.0 2.0
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: vertex
Data variables:
    volcello   (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 2, 35, 1080, 1440), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    0.0
    branch_time_in_parent:   36500.0
    comment:                 <null ref>
    contact:                 gfdl.climate.model.info@noaa.gov
    creation_date:           2019-02-24T05:33:48Z
    data_specs_version:      01.00.27
    experiment:              historical
    experiment_id:           historical
    external_variables:      areacello volcello
    forcing_index:           1
    frequency:               mon
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    grid_label:              gn
    history:                 File was processed by fremetar (GFDL analog of C...
    initialization_index:    1
    institution:             National Oceanic and Atmospheric Administration,...
    institution_id:          NOAA-GFDL
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    mip_era:                 CMIP6
    nominal_resolution:      25 km
    parent_activity_id:      CMIP
    parent_experiment_id:    piControl
    parent_mip_era:          CMIP6
    parent_source_id:        GFDL-CM4
    parent_time_units:       days since 0001-1-1
    parent_variant_label:    r1i1p1f1
    physics_index:           1
    product:                 model-output
    realization_index:       1
    realm:                   ocean
    references:              see further_info_url attribute
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    source_id:               GFDL-CM4
    source_type:             AOGCM
    sub_experiment:          none
    sub_experiment_id:       none
    table_id:                Omon
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    tracking_id:             hdl:21.14100/79122291-5823-4e21-8e3e-0f59a9ff542...
    variable_id:             volcello
    variant_info:            N/A
    variant_label:           r1i1p1f1
    status:                  2019-10-14;created;by nhn2@columbia.edu
    intake_esm_varname:      volcello
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

## 3. Open Cluster 

In [13]:
from dask.distributed import Client
from dask_gateway import Gateway


gateway = Gateway()  # connect to Gateway

cluster = gateway.new_cluster()  # create cluster
cluster.scale(10)  # scale cluster

client = Client(cluster)  # connect Client to Cluster

In [14]:
client

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.c28e14a400e842bda5ce5a349c54a986 Dashboard: https://us-central1-b.gcp.pangeo.io/services/dask-gateway/clusters/prod.c28e14a400e842bda5ce5a349c54a986/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## 4. Actual Calculations

In [12]:
lat1 = -4.49531
lon1 = -208.21236
lat2 = -5.989064
lon2 = -205.234668

In [15]:
isec, jsec, xsec, ysec = sectionate.create_section(grid['lon'], grid['lat'], lon1, lat1, lon2, lat2)

best fit is rounding down


In [29]:
uvpoints = sectionate.transports_C.MOM6_UVpoints_from_section(isec, jsec)
uvdata = np.empty((17,5))

for pt in range(len(uvpoints)):
    point = uvpoints[pt]
    pttype, i, j = point
    i = int(i)
    j = int(j)
    if pttype == 'U':
        lon = grid['lon'].isel(x=i, y=j).values
        lat = grid['lat'].isel(x=i, y=j).values
        letter = 0                               #if pttype is u, then letter is 0
    else:
        lon = grid['lon'].isel(x=i, y=j).values
        lat = grid['lat'].isel(x=i, y=j).values
        letter = 1                               #if pttype is v, then letter is 1
    uvdata[pt] = [letter, i, j, lon, lat]
    print(f'{point[0]}, {point[1]}, {point[2]}, {lon}, {lat}')

U, 367, 485, -208.125, -4.495380878448486
V, 368, 484, -207.875, -4.744568347930908
V, 369, 484, -207.625, -4.744568347930908
U, 369, 484, -207.625, -4.744568347930908
V, 370, 483, -207.375, -4.99366569519043
V, 371, 483, -207.125, -4.99366569519043
U, 371, 483, -207.125, -4.99366569519043
V, 372, 482, -206.875, -5.242669105529785
V, 373, 482, -206.625, -5.242669105529785
U, 373, 482, -206.625, -5.242669105529785
V, 374, 481, -206.375, -5.491572856903076
V, 375, 481, -206.125, -5.491572856903076
U, 375, 481, -206.125, -5.491572856903076
V, 376, 480, -205.875, -5.740372657775879
V, 377, 480, -205.625, -5.740372657775879
U, 377, 480, -205.625, -5.740372657775879
V, 378, 479, -205.375, -5.989063739776611


In [28]:
uvdata[:5]

array([[   0.        ,  367.        ,  485.        , -208.125     ,
          -4.49538088],
       [   1.        ,  368.        ,  484.        , -207.875     ,
          -4.74456835],
       [   1.        ,  369.        ,  484.        , -207.625     ,
          -4.74456835],
       [   0.        ,  369.        ,  484.        , -207.625     ,
          -4.74456835],
       [   1.        ,  370.        ,  483.        , -207.375     ,
          -4.9936657 ]])

In [ ]:
for i in range(len(uvdata)):
    if uvdata[i,0] == 0:    #U-point
        something
    else:                   #V-point
        somethingelse

## Closing Clusters after use

In [17]:
client.close()
cluster.close()